<a href="https://colab.research.google.com/github/Aathib/Repo1/blob/master/stacknet_ieee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aathibala","key":"b629361a9eb4c179a69a6018a9e4e12a"}'}

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import time
from xgboost import XGBClassifier
from hyperopt import hp, tpe, STATUS_OK, Trials, fmin
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from sklearn.preprocessing import StandardScaler
rcParams['figure.figsize'] = 12, 5
import seaborn as sns
from lightgbm import LGBMRegressor, LGBMClassifier
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor, GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.decomposition import PCA

In [5]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c ieee-fraud-detection
!mkdir train
!unzip train_transaction.csv.zip -d train
!unzip test_transaction.csv.zip -d train

 84% 44.0M/52.5M [00:01<00:00, 22.3MB/s]
100% 52.5M/52.5M [00:01<00:00, 38.9MB/s]
  0% 0.00/3.02M [00:00<?, ?B/s]
100% 3.02M/3.02M [00:00<00:00, 100MB/s]
 95% 45.0M/47.3M [00:01<00:00, 24.5MB/s]
100% 47.3M/47.3M [00:01<00:00, 40.8MB/s]
  0% 0.00/2.97M [00:00<?, ?B/s]
100% 2.97M/2.97M [00:00<00:00, 98.8MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 149MB/s]
mkdir: cannot create directory ‘train’: File exists
Archive:  train_transaction.csv.zip
  inflating: train/train_transaction.csv  
Archive:  test_transaction.csv.zip
  inflating: train/test_transaction.csv  


In [0]:
os.chdir('train')
data1 = pd.read_csv('train_transaction.csv')
test1 = pd.read_csv('test_transaction.csv')
data = data1.copy()
test = test1.copy()
data.drop(['P_emaildomain','R_emaildomain', 'TransactionID','TransactionDT'], axis = 1, inplace = True)
test.drop(['P_emaildomain','R_emaildomain', 'TransactionID','TransactionDT'], axis = 1, inplace = True)

In [0]:
rm_cols = ['V'+str(i) for i in range(1,340)]
data.drop(rm_cols, axis = 1, inplace = True)
test.drop(rm_cols, axis = 1, inplace = True)

In [0]:
log_col = ['TransactionAmt', 'dist1', 'dist2'] + ['C%d' % i for i in range(1,15)]
cat_cols  = data.select_dtypes(include='object').columns
numeric_cols  =  [c for c in data.columns if c not in cat_cols]
non_log_cols = [c for c in numeric_cols if c not in log_col]

In [0]:
for c in log_col:
  data[c] = data[c].apply(lambda x: np.log10(x+1e-1))
  test[c] = test[c].apply(lambda x: np.log10(x+1e-1))

In [0]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
data[numeric_cols[1:]] = ss.fit_transform(data[numeric_cols[1:]])
test[numeric_cols[1:]] = ss.transform(test[numeric_cols[1:]])

In [0]:
a = data.isnull().sum()/len(data)*100
variables = data.columns
variable = [ ]
for i in range(0,len(variables)):
    if a[i]<=60:   #setting the threshold as 20%
        variable.append(variables[i])
        
d1 = data[variable]

In [0]:
cat_vars = list(d1.select_dtypes(include='object').columns)
for var in cat_vars:
    cat_list='var'+'_'+ var
    cat_list = pd.get_dummies(d1[var], prefix=var,drop_first =True)
    df2=d1.join(cat_list)
    d1 = df2
data_vars=d1.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
data_final=d1[to_keep]

In [0]:
data_final.drop('card6_debit or credit',inplace = True, axis = 1)

In [0]:
X = data_final.drop('isFraud', axis = 1)
y = data_final['isFraud']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [0]:
testdata = test.copy()
cat_vars_t = list(testdata.select_dtypes(include='object').columns)
for var in cat_vars_t:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(testdata[var], prefix=var,drop_first =True)
    data1=testdata.join(cat_list)
    testdata = data1
        
data_vars=testdata.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]
testdata=testdata[X_train.columns]

In [0]:
X_train.fillna(X_train.mean(), inplace = True)
X_test.fillna(X_test.mean(), inplace = True)
testdata.fillna(testdata.mean(), inplace = True)

In [21]:
!git clone https://github.com/h2oai/pystacknet

Cloning into 'pystacknet'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), done.


In [22]:
cd pystacknet

/content/train/pystacknet


In [0]:
!python setup.py install

In [0]:
models=[ ######## First level ########
            [
             LGBMClassifier(boosting_type='gbdt',
                        objective="binary",
                        metric="AUC",
                    
                        boost_from_average="false",
                        learning_rate=0.0045,
                        num_leaves=491,
                        max_depth=20,
                        min_child_weight=0.035,
                        feature_fraction=0.38,
                        bagging_fraction=0.42,
                        min_data_in_leaf=100,
                        max_bin=255,
                        importance_type='split',
                        reg_alpha=0.4,
                        reg_lambda=0.65,
                        bagging_seed=1,
                        random_state=1,
                        verbosity=-1,
                        subsample=0.85,
                        colsample_bytree=0.8,
                        min_child_samples=79),
             CatBoostClassifier(learning_rate=0.2,
                            bagging_temperature=0.1, 
                            l2_leaf_reg=30,
                            depth=12, 
                            max_bin=255,
                            iterations=100,
                            loss_function='Logloss',
                            objective='Logloss',
                            eval_metric="AUC",
                            bootstrap_type='Bayesian',
                            random_seed=1,
                            task_type = "GPU" ,
                            early_stopping_rounds=10),
             XGBClassifier(reg_alpha = 0, colsample_bytree = 0.75,eta = 0.125,gamma = 0.60,reg_lambda = 1.7,max_depth = 12,min_child_weight = 1.,n_estimators = 600,subsample =0.95 ,tree_method = 'gpu_hist',
                           objective= 'binary:logistic',max_delta_step  = 1,seed=1)      
            ],
            ######## Second level ########
            [RandomForestClassifier (n_estimators=500, criterion="entropy", max_depth=10, random_state=1),
            ]]

In [0]:
from pystacknet.pystacknet import StackNetClassifier

model=StackNetClassifier(models, metric="auc", folds=4,
	restacking=False,use_retraining=True, use_proba=True, 
	random_state=12345,n_jobs=1, verbose=1)

model.fit(X_train,y_train)

In [0]:
y_pred = model.predict_proba(X_test)

y_pred = (y_pred>0.6).astype(int)
data = pd.DataFrame(y_pred, columns = ['False', 'True'])

In [0]:
data['new']= 0
for i in range(len(data)):
  if data['False'][i] ==0:
    data['new'][i]= 1
  else:
    data['new'][i] = 0

y_pred = data['new']

predictions = [value for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
AUC = roc_auc_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("AUC: %.2f%%" % (AUC * 100.0))
print(classification_report(y_test, predictions))

In [30]:
!nvidia-smi

Tue Sep 10 07:29:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  